In [1]:
import numpy as np
import openpnm as op
import porespy as ps
import matplotlib.pyplot as plt
np.set_printoptions(precision=4)
np.random.seed(10)
from mpl_toolkits import mplot3d
%matplotlib inline
ps.visualization.set_mpl_style()
import scipy.ndimage as spim
import matplotlib.pyplot as plt
from porespy.filters import find_peaks, trim_saddle_points, trim_nearby_peaks
from porespy.tools import randomize_colors
from skimage.segmentation import watershed

ps.settings.tqdm['disable'] = True
ps.visualization.set_mpl_style()
import imageio

In [2]:
resolution = 2.25e-6
name = 'Bentheimer'

In [4]:
raw_file = np.fromfile(name+'.raw', dtype=np.uint8)
im = (raw_file.reshape(1000,1000,1000))
im = im==0;

In [5]:
im=im[:500,:500,:500]

In [12]:
net = ps.networks.snow2(im, voxel_size=resolution)

In [13]:
pn = op.io.network_from_porespy(net.network)
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Network at 0x22abf17ddb0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  throat.conns                                                10663 / 10663
  3  pore.coords                                                   6466 / 6466
  4  pore.region_label                                             6466 / 6466
  5  pore.phase                                                    6466 / 6466
  6  throat.phases                                               10663 / 10663
  7  pore.region_volume                                            6466 / 6466
  8  pore.equivalent_diameter                                      6466 / 6466
  9  pore.local_peak                                               6466 / 6466
 1

In [18]:
pn.add_model_collection(op.models.collections.geometry.cubes_and_cuboids)
pn.regenerate_models()
op.topotools.label_faces(pn, label='surface')

In [20]:
h = op.utils.check_network_health(pn)
op.topotools.trim(network=pn, pores=h['isolated_pores'])
print(h)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Key                                 Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
headless_throats                    []
looped_throats                      []
isolated_pores                      []
disconnected_pores                  [0, 8, 12, 22, 25, 138, 139, 170, 183, 184, 188, 189, 194, 198, 215, 217, 218, 230, 247, 250, 269, 282, 286, 302, 311, 316, 342, 351, 352, 353, 365, 375, 377, 384, 407, 412, 417, 430, 431, 439, 450, 451, 459, 460, 462, 470, 477, 491, 497, 529, 543, 552, 553, 574, 586, 587, 603, 604, 605, 616, 617, 618, 635, 636, 644, 646, 647, 656, 657, 658, 659, 672, 673, 674, 683, 693, 694, 695, 707, 727, 749, 760, 775, 776, 783, 791, 799, 823, 923, 924, 931, 960, 980, 990, 1021, 1044, 1057, 1130, 1137, 1191, 1200, 1213, 1233, 1235, 1252, 1280, 1291, 1388, 1402, 1485, 1588, 1660, 1995, 2012, 2013, 2050, 2089, 2123, 2211, 2212, 2220, 2232, 2340, 2350, 235

In [22]:
h = op.utils.check_network_health(pn)
op.topotools.trim(network=pn, pores=h['disconnected_pores'])
print(h)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Key                                 Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
headless_throats                    []
looped_throats                      []
isolated_pores                      []
disconnected_pores                  []
duplicate_throats                   []
bidirectional_throats               []
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


In [23]:
print(pn)


══════════════════════════════════════════════════════════════════════════════
net : <openpnm.network.Network at 0x22abf17ddb0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  2  throat.conns                                                10478 / 10478
  3  pore.coords                                                   5895 / 5895
  4  pore.region_label                                             5895 / 5895
  5  pore.phase                                                    5895 / 5895
  6  throat.phases                                               10478 / 10478
  7  pore.region_volume                                            5895 / 5895
  8  pore.equivalent_diameter                                      5895 / 5895
  9  pore.local_peak                                               5895 / 5895
 1

In [24]:
Vol_void = np.sum(pn['pore.volume'])+np.sum(pn['throat.volume'])
inlet = pn.pores('left')
outlet = pn.pores('right')
A = op.topotools.get_domain_area(pn, inlets=inlet, outlets=outlet)
L = op.topotools.get_domain_length(pn, inlets=inlet, outlets=outlet)
Vol_bulk = A * L
Poro = Vol_void / Vol_bulk
print(f'The value of Porosity is: {Poro:.2f}')


[22:42:43] WARNING  Attempting to estimate inlet area...will be low                              ]8;id=400098;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py\_topotools.py]8;;\:]8;id=623282;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py#1033\1033]8;;\

           ERROR    Inlet and outlet faces are different area                                    ]8;id=275094;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py\_topotools.py]8;;\:]8;id=45077;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py#1051\1051]8;;\

           WARNING  Attempting to estimate domain length...could be low if boundary pores were   ]8;id=813610;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py\_topotools.py]8;;\:]8;id=130787;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py#1077\1077]8;;\
                    not added                                                                                      

           ERROR    A unique value of length could not be found                                  ]8;id=908640;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py\_topotools.py]8;;\:]8;id=390363;file://c:\Users\Sabda\anaconda3\envs\pg\lib\site-packages\openpnm\topotools\_topotools.py#1087\1087]8;;\

The value of Porosity is: 0.08


In [25]:
pn.add_model_collection(op.models.collections.geometry.spheres_and_cylinders)
pn.regenerate_models()

In [26]:
Vol_void_initial = np.sum(pn['pore.volume'])+np.sum(pn['throat.total_volume'])
Vol_void_corrected = np.sum(pn['pore.volume'])+np.sum(pn['throat.volume'])
Poro_initial = Vol_void_initial / Vol_bulk
Poro_corrected = Vol_void_corrected / Vol_bulk
print(f'Initial Porosity: {Poro_initial:.5f}')
print(f'Corrected Porosity: {Poro_corrected:.5f}')

Initial Porosity: 0.04603
Corrected Porosity: 0.04567


In [33]:
im_poro = ps.metrics.porosity(im)
print(f"Porosity from image: {im_poro*100:.1f}%")

Porosity from image: 21.2%


In [28]:
snow = ps.networks.snow2(im, boundary_width = 0)
network = snow.network
pn = op.io.network_from_porespy(network)
pn['pore.diameter']=network['pore.inscribed_diameter']
pn['throat.diameter']=network['throat.inscribed_diameter']
model=op.models.geometry.throat_length.cubes_and_cuboids
pn.add_model(propname='throat.length',
             model=model,
             regen_mode='normal')
model=op.models.geometry.pore_volume.cube
pn.add_model(propname='pore.volume',
             model=model,
             regen_mode='normal')
model=op.models.geometry.throat_volume.cuboid
pn.add_model(propname='throat.volume',
             model=model,
             regen_mode='normal')
pn.regenerate_models()

In [32]:
Vol_void = np.sum(pn['pore.volume'])+np.sum(pn['throat.volume'])
Vol_bulk = 500**3 #from dimensionality data
pnm_poro = Vol_void / Vol_bulk
print(f"Porosity from pnm: {pnm_poro*100:.1f}%")

Porosity from pnm: 15.0%
